In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tslearn.clustering import KShape
from sklearn.metrics import silhouette_score

# --- 데이터 로드 ---
print("Loading preprocessed data...")
X_train = np.load('./dataset/preprocessed_training_dataset.npy')

c:\Users\ckh06\anaconda3\envs\mesp\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


Loading preprocessed data...


In [5]:
# --- 단일 K값에 대한 계산을 수행하는 함수 (병렬 작업을 위해 분리) ---
def calculate_kshape_metrics(k, data):
    """하나의 k 값에 대해 K-Shape 모델을 학습하고 inertia와 silhouette 점수를 반환"""
    try:
        model = KShape(
            n_clusters=k,
            n_init=3,
            random_state=42,
            verbose=-1 # 병렬 처리 시 로그가 섞이지 않도록 비활성화
        )
        labels = model.fit_predict(data)
        
        inertia = model.inertia_
        
        # 실루엣 점수는 클러스터가 2개 이상일 때만 의미 있음
        if len(np.unique(labels)) > 1:
            distances = model.transform(data)
            score = silhouette_score(distances, labels, metric='euclidean')
        else:
            score = -1 # 계산 불가 시 -1
            
        return (k, inertia, score)
    except Exception as e:
        print(f"Error calculating for k={k}: {e}")
        return (k, None, None)

In [6]:
from joblib import Parallel, delayed

# --- 병렬 처리로 최적 K 탐색 실행 ---
k_range = range(2, 16)
print(f"Finding optimal K for K-Shape in parallel for k={min(k_range)} to {max(k_range)}...")

# n_jobs=-1: 모든 CPU 코어를 사용
# 각 코어는 k_range에서 k값을 하나씩 가져가 calculate_kshape_metrics 함수를 독립적으로 실행
results = Parallel(n_jobs=-1)(
    delayed(calculate_kshape_metrics)(k, X_train) for k in tqdm(k_range, desc="Dispatching K-value jobs")
)

Finding optimal K for K-Shape in parallel for k=2 to 15...


Dispatching K-value jobs: 100%|██████████| 14/14 [00:00<00:00, 248.53it/s]


KeyboardInterrupt: 

In [ ]:
# --- 결과 취합 및 시각화 ---
print("\nProcessing and visualizing results...")

# 병렬 처리는 순서를 보장하지 않으므로 k값을 기준으로 결과를 정렬
results.sort(key=lambda x: x[0])

# None 값을 제외하고 유효한 결과만 추출
valid_results = [res for res in results if res[1] is not None and res[2] is not None]

# 정렬된 결과에서 k, inertia, silhouette 점수를 각각 분리
k_values = [res[0] for res in valid_results]
inertia_values = [res[1] for res in valid_results]
silhouette_scores = [res[2] for res in valid_results]

# 결과 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle('Optimal K for K-Shape (Parallel Computation)', fontsize=16)

# 엘보우 기법 그래프
ax1.plot(k_values, inertia_values, 'bo-')
ax1.set_xlabel('Number of clusters (K)')
ax1.set_ylabel('Inertia')
ax1.set_title('Elbow Method')
ax1.grid(True)

# 실루엣 점수 그래프
ax2.plot(k_values, silhouette_scores, 'ro-')
ax2.set_xlabel('Number of clusters (K)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score')
ax2.grid(True)

plt.show()